# Bootstrapping user preferences



Similarly to (Zhu et al., 2021), we'll evaluate a dynamic recommendation system using:

1. An oracle preference model (Generated in notebook 00-preference_model)
2. A position bias examination model
3. Bootstrapped user preferences

Step 3 is important so that we're simulating exposing users to fresh items, using their previous preferences (provided by step 1) and their examination bias (step 2) to determine a sinthethical rating matrix dataset that we'll use to compare different calibration approaches as the user gets exposed to more and more items.

In [57]:
import pandas as pd
from tqdm import tqdm

In [58]:
import numpy as np
import math
import random

In [59]:
import sys
import os
from pathlib import Path

sys.path.append('..')

In [60]:
from constants import ML_1M_FILLED_PATH

In [61]:
def get_user_preference_for_item(user, item, matrix):
    user_ratings = matrix[matrix["user"] == user]
    return user_ratings[user_ratings["item"] == item].rating.item()

In [62]:
def click_model(k):
    lambda_k = 1/math.log(k+1,2)
    examination_probability = random.random()
    if examination_probability <= lambda_k:
        return True
    return False

In [63]:
def get_inverse_propensity_click_score(position):
    # Given a click position, this funtion returns the invense propensity, 
    # usefull to debias the data later.
    return - 1/math.log(position+1,2)

In [64]:
def get_user_feedback_for_item(user, item ,k, matrix):
    preference = get_user_preference_for_item(user, item, matrix)
    observed = click_model(k)
    relevant = bool(preference)
    should_click = observed and relevant
    if (should_click):
        feedback = 1
        clicked_at = k
    else:
        if (observed):
            # Case where an item was observed, but isn´t relevant -> negative example for BPR
            feedback = 0
        else:
            # Case where an item was neither observed or relevant -> we will ignore this training instance in this loop
            feedback = None
        clicked_at = None
        
    # If user clicked the item, record the position it was in
    # feedback = 1 if user examined and clicked, 0 if user examined and not clicked,
    # None if otherwise
    return user, item, feedback, clicked_at

In [65]:
def map_recommendation_to_feedback(user, rec_list, matrix):
    return [get_user_feedback_for_item(user, item, idx+1, matrix) for idx, item in enumerate(rec_list)]

In [66]:
def random_rec(items, u, k, D):
    user_history = set(D[D["user"] == u]["item"])
    candidate_items = list(set(items) - user_history)
    return random.sample(candidate_items, k)

In [67]:
preference_matrix = pd.read_csv(f"../{ML_1M_FILLED_PATH}").drop(columns=["Unnamed: 0"])

In [68]:
unique_users = list(preference_matrix["user"].unique())

unique_items = list(preference_matrix["item"].unique())

In [69]:
click_matrix = pd.DataFrame(columns=["user", "item", "clicked_and_examined", "clicked_at"])

In [70]:
def bootstrap_clicks(D, unique_users, unique_items, preference_matrix, k=20, rounds=10):
    """
    Given unique users and unique items, recommend up to k items to every user
    using a preference matrix as a relevancy model and using a click model
    to simulate probability of user examinating an item.

    Feedback signal will be fed to the D matrix.

    We run the boostrap process for a total of an arbitrary number of rounds,
    in order to ensure enough feedback data to train a model.
    """
    new_df = D.copy()#pd.DataFrame(columns=["user", "item", "clicked_and_examined", "clicked_at"])
    for round in range(rounds):
        rows_to_append = []
        for user in tqdm(unique_users, desc=f"Processing users (round {round+1}/{rounds})..."):
            recs = random_rec(unique_items, user, k, new_df)
            row = map_recommendation_to_feedback(user, recs, preference_matrix)
            rows_to_append.extend(row)
        round_df = pd.DataFrame(rows_to_append, columns=new_df.columns)
        new_df = pd.concat([new_df, round_df], ignore_index=True)
    return pd.concat([D, new_df])

In [ ]:
%%time
click_matrix = bootstrap_clicks(click_matrix, unique_users, unique_items, preference_matrix)

Processing users (round 1/10)...: 100%|██████████| 6040/6040 [18:02<00:00,  5.58it/s]
/tmp/ipykernel_7989/1541821497.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, round_df], ignore_index=True)
Processing users (round 6/10)...:  43%|████▎     | 2575/6040 [08:45<12:02,  4.80it/s]

In [ ]:
click_matrix

,user,item,relevancy,clicked_at
0,1,3183,1.0,1.0
1,1,1375,NaN,NaN
2,1,2162,NaN,NaN
3,1,353,NaN,NaN
4,1,2519,NaN,NaN
...,...,...,...,...
120795,6040,3248,NaN,NaN
120796,6040,3937,0.0,NaN
120797,6040,3423,NaN,NaN
120798,6040,3246,NaN,NaN


In [ ]:
click_matrix.groupby(["relevancy", "clicked_at"], dropna=False).size()

relevancy  clicked_at
0.0        NaN           28588
1.0        1.0            1988
           2.0            1251
           3.0             960
           4.0             842
           5.0             745
           6.0             735
           7.0             652
           8.0             617
           9.0             560
           10.0            540
           11.0            573
           12.0            518
           13.0            508
           14.0            519
           15.0            481
           16.0            433
           17.0            492
           18.0            481
           19.0            423
           20.0            455
NaN        NaN           78439
dtype: int64

In [ ]:
click_matrix.to_csv("../data/simulation/sinthetic_data.csv", index=False)

In [ ]:
click_matrix = pd.read_csv("../data/simulation/sinthetic_data.csv")

In [ ]:
click_matrix[click_matrix["relevancy"].notna()]["relevancy"].value_counts(dropna=False)

relevancy
0.0    28588
1.0    13773
Name: count, dtype: int64